In [ ]:
# 模块导入
import numpy as np;
import pandas as pd;
import tensorflow as tf;
from tensorflow.keras import layers;
from matplotlib import pyplot as plt;

# 设置数据展示格式
pd.options.display.max_rows = 10;
pd.options.display.float_format = "{:.1f}".format;
tf.keras.backend.set_floatx('float32');

# 读取并打乱数据
train_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
test_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")
train_df = train_df.reindex(np.random.permutation(train_df.index)); # shuffle the training set

# normalize values (将各个属性的数据控制在一定范围内，比如 -3～3)，方便训练模型
train_df_mean = train_df.mean();
train_df_std = train_df.std();
train_df_norm = (train_df - train_df_mean)/train_df_std;

# Examine some of the values of the normalized training set. Notice that most 
# Z-scores fall between -2 and +2.
# head 方法取前五行数据
print(train_df_norm.head());

# 对测试集数据做同样操作
test_df_mean = test_df.mean();
test_df_std = test_df.std();
test_df_norm = (test_df - test_df_mean)/test_df_std;

# create binary label
threshold = 265000;
train_df_norm["median_house_value_is_high"] = (train_df["median_house_value"] > threshold).astype(float);
test_df_norm["median_house_value_is_high"] = (test_df["median_house_value"] > threshold).astype(float);
train_df_norm["median_house_value_is_high"].head(8000)

# 除此之外，还可以基于 norm 后的数据设置 threshold 和 median_hourse_valud_is_high

# threshold_in_Z = 1.0 
# train_df_norm["median_house_value_is_high"] = (train_df_norm["median_house_value"] > threshold_in_Z).astype(float)
# test_df_norm["median_house_value_is_high"] = (test_df_norm["median_house_value"] > threshold_in_Z).astype(float) 

# represent feature in feature columns
feature_columns = [];

median_income = tf.feature_column.numeric_column("median_income");
feature_columns.append(median_income);

tr = tf.feature_column.numeric_column("total_rooms");
feature_columns.append(tr);

feature_layer = layers.DenseFeatures(feature_columns);

feature_layer(dict(train_df_norm));

# create a simple classification model
def create_model(my_learning_rate, feature_layer, my_metrics):
    model = tf.keras.models.Sequential();
    
    model.add(feature_layer);
    
    # units 和 input_shape 为什么是 1 ？
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,), activation=tf.sigmoid),);
    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = my_learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(),
                  metrics = my_metrics);
    
    return model;

def train_model(model, dataset, epochs, label_name, batch_size = None, shuffle = True):
    features = {name: np.array(value) for name, value in dataset.items()};
    label = np.array(features.pop(label_name));
    history = model.fit(x=features, y=label, batch_size=batch_size, epochs=epochs, shuffle=shuffle);
    
    epochs = history.epoch;
    
    hist = pd.DataFrame(history.history);
    
    return epochs, hist;

# plot the relation between epoch and one or more classification metrics
def plot_curve(epochs, hist, list_of_metrics):
    plt.figure();
    plt.xlabel("Epoch");
    plt.ylabel("Value");
    
    for m in list_of_metrics:
        x = hist[m];
        plt.plot(epochs[1:], x[1:], label=m);
    
    plt.legend();

# main function
learning_rate = 0.001;
epochs = 20;
batch_size = 100;
label_name = "median_house_value_is_high";
classification_threshold = 0.52; # metrics 的 cut point

# Establish the metrics the model will measure.
METRICS = [
#            tf.keras.metrics.BinaryAccuracy(name='accuracy', 
#                                            threshold=classification_threshold),
#            tf.keras.metrics.Precision(name='precision', thresholds=classification_threshold),
#            tf.keras.metrics.Recall(name='recall', thresholds=classification_threshold),
            tf.keras.metrics.AUC(num_thresholds=100, name='auc'),
          ];

my_model = create_model(learning_rate, feature_layer, METRICS);

epochs, hist = train_model(my_model, train_df_norm, epochs, label_name, batch_size, True);

list_of_metrics_to_plot = ["auc"];

plot_curve(epochs, hist, list_of_metrics_to_plot);